#Stock Indicators Python

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279190,85.324337,83.909667,84.556805,118944600
1,2010-01-05,85.504951,85.542578,84.918014,85.226535,111579900
2,2010-01-06,85.565140,85.775835,85.354445,85.422166,116074400
3,2010-01-07,85.926353,86.031701,85.166341,85.407136,131091100
4,2010-01-08,86.212280,86.249907,85.527521,85.700590,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
df['bullishMACD'] = 0.0
df['bullishMACD'] = np.where(df['macd'] > df['signal'], 1.0, 0.0)
df['crossover_MACD'] = df['bullishMACD'].diff()

# Williams Alligator
df['jaw'] = [r.jaw for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['teeth'] = [r.teeth for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['lips'] = [r.lips for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]

# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Stochastic Momentum Index (SMI)
df['smi'] = [r.smi for r in indicators.get_smi(quotes, 14, 3, 3)]
df['smi_signal'] = [r.signal for r in indicators.get_smi(quotes, 14, 3, 3)]
df['bullishSMI'] = 0.0
df['bullishSMI'] = np.where(df['smi'] > df['smi_signal'], 1.0, 0.0)
df['crossover_SMI'] = df['bullishSMI'].diff()   

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]
df['chandelier_exit'] = 0
df['chandelier_exit'] = np.where(df['chandelier_long_exit'] > df['chandelier_short_exit'], 1.0, 0.0)
df['crossover_chandelier_exit'] = df['chandelier_exit'].diff()  

# Calculate Bollinger Bands
df['BB_upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]

# Arnaud Legoux Moving Average (ALMA)
df['ALMA_21'] = [r.alma for r in indicators.get_alma(quotes, 21, 0.85, 6)]
df['ALMA_50'] = [r.alma for r in indicators.get_alma(quotes, 50, 0.85, 6)]
df['alma_bullish'] = 0.0
df['alma_bullish'] = np.where(df['ALMA_21'] > df['ALMA_50'], 1.0, 0.0)
df['crossover_ALMA'] = df['alma_bullish'].diff()


# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]


In [5]:
df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,macd,signal,histogram,bullishMACD,...,BB_middle_band,BB_lower_band,ALMA_21,ALMA_50,alma_bullish,crossover_ALMA,rsi,rsima6,rsima12,dynamic20
0,2010-03-16,87.596863,87.679631,86.904572,87.145367,168673000,0.986044,0.594635,0.391409,1.0,...,84.854050,81.757991,86.387412,84.698008,1.0,1.0,74.335825,70.677619,66.191693,84.793565
1,2010-03-17,88.116081,88.402030,87.604391,87.860239,177468100,1.083932,0.692494,0.391437,1.0,...,85.111401,81.834480,86.700350,84.981462,1.0,0.0,76.478378,72.177486,67.757279,85.030984
2,2010-03-18,88.070908,88.243976,87.717238,88.123581,196509100,1.144668,0.782929,0.361739,1.0,...,85.342036,81.917046,86.984447,85.257446,1.0,0.0,75.884799,73.276955,69.171193,85.251105
3,2010-03-19,87.625130,88.622501,87.285113,87.625130,226641100,1.143649,0.855073,0.288576,1.0,...,85.541729,82.073558,87.197263,85.508036,1.0,0.0,70.102608,73.139524,70.069121,85.428356
4,2010-03-22,88.093613,88.252291,87.073575,87.126465,184477800,1.167189,0.917496,0.249693,1.0,...,85.764093,82.240694,87.410514,85.765384,1.0,0.0,72.476200,73.391910,71.045474,85.624777


In [6]:
# Moving Average Convergence / Divergence (MACD)
if __name__ == '__main__':
    
    chart = Chart(title="MACD Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    macd_line = macd_chart.create_line('macd', color="#2ee30f", width=1, price_line=False, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('histogram', color="#1341e4", price_line=False, price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])


    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        MACD_diff = df.iloc[i]['crossover_MACD']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if MACD_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif MACD_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [7]:
# Williams Alligator
if __name__ == '__main__':
    
    chart = Chart(title="Williams Alligator", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    jaw_line = chart.create_line('jaw', color="#1f77b4", width=1, price_line=False, price_label=False)
    jaw_line.set(df[['Date', 'jaw']])
    teeth_line = chart.create_line('teeth', color="#ff7f0e", width=1, price_line=False, price_label=False)
    teeth_line.set(df[['Date', 'teeth']])
    lips_line = chart.create_line('lips', color="#2ca02c", width=1, price_line=False, price_label=False)
    lips_line.set(df[['Date', 'lips']])

chart.show(block = True)

In [8]:
# Bollinger Bands
if __name__ == '__main__':

    chart = Chart(title="Bollinger Bands", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_line=False, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_line=False, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_line=False, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])

chart.show(block = True)

In [9]:
# Stochastic Momentum Index (SMI)
if __name__ == '__main__':
    
    chart = Chart(title="Stochastic Momentum Index (SMI)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    smi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    smi_line = smi_chart.create_line('smi', color="#2ee30f", width=1, price_line=False, price_label=False)
    smi_line.set(df[['Date', 'smi']])

    signal_line = smi_chart.create_line('smi_signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'smi_signal']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        SMI_diff = df.iloc[i]['crossover_SMI']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if SMI_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif SMI_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [10]:
# Chandelier Exit

if __name__ == '__main__':
    
    chart = Chart(title="chandelier long_exit and short_exit Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Add Chandelier Exit lines
    chandelier_long_line = chart.create_line('Chandelier_Long_Exit', color='#8e44ad', width=1, price_line=False, price_label=False)
    chandelier_long_line.set(df[['Date', 'chandelier_long_exit']])
    # chandelier_short_line = chart.create_line('Chandelier_Short_Exit', color='#e74c3c', width=1, price_line=False, price_label=False)
    # chandelier_short_line.set(df[['Date', 'chandelier_short_exit']])

    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        chan_diff = df.iloc[i]['crossover_chandelier_exit']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if chan_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif chan_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [16]:
# Now, let's create the chart with lightweight-charts and add the EMAs and buy/sell markers.

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    chart = Chart(title="EMA12_EMA25 Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_line=False, price_label=False)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_line=False, price_label=False)
    ema25_line.set(df[['Date', 'EMA 25']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# Arnaud Legoux Moving Average (ALMA)

if __name__ == '__main__':
    
    chart = Chart(title="Arnaud Legoux Moving Average (ALMA)", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    alma21_line = chart.create_line('ALMA_21', color='#ffeb3b', width=1, price_line=False, price_label=False)
    alma21_line.set(df[['Date', 'ALMA_21']])

    alma50_line = chart.create_line('ALMA_50', color='#26c6da', width=1, price_line=False, price_label=False)
    alma50_line.set(df[['Date', 'ALMA_50']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        alma_diff = df.iloc[i]['crossover_ALMA']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if alma_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif alma_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
chart.layout()

AttributeError: 'Chart' object has no attribute 'theme'